In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@stretaildataproj.dfs.core.windows.net/products")

In [0]:
df = df.drop("_rescued_data")
df.display()

In [0]:
df.createOrReplaceTempView("products")

In [0]:
%sql
select * from products


In [0]:
%sql
CREATE OR REPLACE FUNCTION retail_databricks_catalog.bronze.discount_function(para_price DOUBLE)
RETURNS DOUBLE
LANGUAGE SQL
RETURN para_price * 0.90 

In [0]:
%sql
SELECT product_id,
       price,
       retail_databricks_catalog.bronze.discount_function(price) AS discounted_price
FROM products;
